## Vector Search
We need to take our deployed vector search and prepare a documented tool for Agent usage

In [0]:
%sql
CREATE OR REPLACE FUNCTION shm.noc_agent.vector_search(
  occupation_description STRING
)
RETURNS TABLE (
  noc_result STRING
)
COMMENT 'Returns the National Occupational Classification (NOC) code for the provided occupation description. Format the input to include the best broad category from the following list:

0 Legislative and senior management occupations
1 Business, finance and administration occupations
2 Natural and applied sciences and related occupations
3 Health occupations
4 Occupations in education, law and social, community and government services
5 Occupations in art, culture, recreation and sport
6 Sales and service occupations
7 Trades, transport and equipment operators and related occupations
8 Natural resources, agriculture and related production occupations
9 Occupations in manufacturing and utilities

The response will provide the categories, groups, and pages from the NOC guidance.
'
RETURN
SELECT 
CONCAT(
  'Broad Category: ', COALESCE(h1, ''),  
  '\n Major Group: ', COALESCE(h2, 'None'), 
  '\n Sub Group: ', COALESCE(h3, 'None'), 
  '\n Minor Group: ', COALESCE(h4, 'None'), 
  '\n Unit Group: ', COALESCE(h5, 'None'), 
  '\n Pages:', pages
) as noc_result
FROM vector_search(
  index=>'shm.noc_agent.noc_index',
  query_text=>occupation_description,
  num_results=>5
)

In [0]:
%sql
SELECT * FROM shm.noc_agent.vector_search('Petroleum Engineer')

## Employment By Occupation

This notebook preps four functions for analyzing tables based on retrieved national occupational classication

In [0]:
%sql
SELECT * 
FROM shm.noc_agent.monthly_adjusted_employment_by_occupation
LIMIT 10

In [0]:
%sql
CREATE OR REPLACE FUNCTION shm.noc_agent.get_occupation_employment(
  occupation_name STRING,
  months_lookback INT DEFAULT 6
)
RETURNS TABLE (
  date_year_month STRING,
  noc_title STRING,
  employees_thousands DOUBLE
)
COMMENT 'Returns monthly adjusted employment data for occupations matching the provided occupation name. The occupation MUST be a value from the following list, matched based on occupation type, including bracketed numbers ([72-75]):

Health occupations, except management [31-33]
Business, finance and administration occupations, except management [11-14]
Trades, transport and equipment operators and related occupations, except management [72-75]
Management occupations [00,10,20,30,40,50,60,70,80,90]
Natural resources, agriculture and related production occupations, except management [82-85]
Natural and applied sciences and related occupations, except management [21-22]
Occupations in art, culture, recreation and sport, except management [51-55]
Sales and service occupations, except management [62-65]
Total employed, all occupations [00-95]
Occupations in manufacturing and utilities, except management [92-95]
Occupations in education, law and social, community and government services, except management [41-45]
'
RETURN
SELECT
  ref_date as date_year_month,
  national_occupational_classification_noc as noc_title,
  value as employees_thousands
FROM shm.noc_agent.monthly_adjusted_employment_by_occupation
WHERE statistics = 'Estimate'
AND national_occupational_classification_noc = occupation_name
AND to_date(ref_date) >= date_sub(current_date(), CAST((months_lookback+1)*30 AS INT))

## Hours and Wages

In [0]:
%sql
CREATE OR REPLACE FUNCTION shm.noc_agent.get_occupation_wages(
  occupation_name STRING,
  months_lookback INT DEFAULT 6
)
RETURNS TABLE (
  date_year_month STRING,
  noc_title STRING,
  weekly_wages DOUBLE
)
COMMENT 'Returns average weekly wages for occupations matching the provided occupation name. The occupation MUST be a value from the following list, matched based on occupation type, including bracketed numbers ([72-75]):

Health occupations, except management [31-33]
Business, finance and administration occupations, except management [11-14]
Trades, transport and equipment operators and related occupations, except management [72-75]
Management occupations [00,10,20,30,40,50,60,70,80,90]
Natural resources, agriculture and related production occupations, except management [82-85]
Natural and applied sciences and related occupations, except management [21-22]
Occupations in art, culture, recreation and sport, except management [51-55]
Sales and service occupations, except management [62-65]
Total employed, all occupations [00-95]
Occupations in manufacturing and utilities, except management [92-95]
Occupations in education, law and social, community and government services, except management [41-45]
'
RETURN
SELECT
  ref_date as date_year_month,
  characteristics as noc_title,
  value as employees_thousands
FROM shm.noc_agent.monthly_hours_and_wages
WHERE characteristics REGEXP '\\['
AND hours_and_wages = 'Full-time employees, average weekly wages'
AND characteristics = occupation_name
AND to_date(ref_date) >= date_sub(current_date(), CAST((months_lookback+1)*30 AS INT))

In [0]:
%sql
SELECT * FROM shm.noc_agent.get_occupation_wages(
  'Health occupations, except management [31-33]', 48
  )

## What's Next?
We have two harder tables where there is a mix of categories (Job Tenure By Occupation and Monthly Labour Force Characteristics). In order to use these tables, we might need to do some data augmentation and use AI based matching to quickly prepare the table. We could also use Levenshtein Distance to do fuzzy matching based on edit distance from the third level heading.

In [0]:
%sql
SELECT DISTINCT(national_occupational_classification_noc)
FROM shm.noc_agent.monthly_job_tenure_by_occupation
LIMIT 20

In [0]:
%sql
SELECT DISTINCT(national_occupational_classification_noc)
FROM shm.noc_agent.monthly_labour_force_characteristics
LIMIT 20